# Storing Data Frames

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [2]:
myspark=SparkSession.builder.appName("Spark_DF_Operations").master("yarn")\
    .config("spark.executor.memory","4g")\
    .config("hive.exec.dynamic.partition", "true")\
    .config("hive.exec.dynamic.partition.mode", "nonstrict")\
    .enableHiveSupport().getOrCreate()

## Write in HDFS

The data files used below is available in this repo under data folder. LOAD this file in the HDFS at /tmp directory

### 1. Using df.write.save()

df.write.save(path=None, format=None, mode=None, partitionBy=None, **options):

    path – the path in a Hadoop supported file system
    format – the format used to save. options are 'orc','parquet','csv','text'
    mode – specifies the behavior of the save operation when data already exists.
        append: Append contents of this DataFrame to existing data.
        overwrite: Overwrite existing data.
        ignore: Silently ignore this operation if data already exists.
        error (default case): Throw an exception if data already exists.
    partitionBy – names of partitioning columns

In [3]:
df = myspark.read.format("csv").options(header=True, inferSchema=True, sep=",",
    dateFormat="yyyy-MM-dd", timestampFormat="yyyy-MM-dd HH:mm:ss", ignoreLeadingWhiteSpace=True,\
    ignoreTrailingWhiteSpace=True, path="/tmp/sampledata.csv").load()

In [4]:
df = df.withColumn("dated", F.to_date("dated").cast(T.DateType()))\
 .withColumn("timing", F.from_unixtime(F.unix_timestamp("timing"),"yyyy-MM-dd HH:mm:ss"))

In [5]:
df.show(10,False)

+------+------+---+------+----------+-------------------+
|fname |lname |age|height|dated     |timing             |
+------+------+---+------+----------+-------------------+
|naresh|jangra|30 |170.5 |2013-10-12|2013-10-12 12:35:50|
|ravi  |verma |35 |155.67|2014-10-12|2014-10-12 01:55:50|
|viren |nain  |55 |160.0 |2015-10-12|2015-10-12 09:15:50|
|bhanu |pratap|11 |180.8 |2016-10-12|2016-10-12 10:05:50|
+------+------+---+------+----------+-------------------+



In [6]:
# By Default, 200 Partitions will be created after shuffle and it will write 200 files in HDFS. 
# To Control this, set below property.
myspark.conf.set("spark.sql.shuffle.partitions",5)

df.write.save(path = "/tmp/sparkdata", format="csv", mode="overwrite", partitionBy=("age"))

    [768019@pxlbig14 Production EdgeNode 19:21:21 ~]$ hadoop fs -ls /tmp/sparkdata
    Found 5 items
    -rw-rw-rw-   3 768019 supergroup          0 2018-02-25 19:21 /tmp/sparkdata/_SUCCESS
    drwxrwxrwx   - 768019 supergroup          0 2018-02-25 19:21 /tmp/sparkdata/age=11
    drwxrwxrwx   - 768019 supergroup          0 2018-02-25 19:21 /tmp/sparkdata/age=30
    drwxrwxrwx   - 768019 supergroup          0 2018-02-25 19:21 /tmp/sparkdata/age=35
    drwxrwxrwx   - 768019 supergroup          0 2018-02-25 19:21 /tmp/sparkdata/age=55


### 2. df.write.{format}()

df.write.(**options)

    csv(path, mode=None, compression=None, sep=None, quote=None, escape=None, header=None, nullValue=None, escapeQuotes=None, quoteAll=None, dateFormat=None, timestampFormat=None, ignoreLeadingWhiteSpace=None, ignoreTrailingWhiteSpace=None)

    json(path, mode=None, compression=None, dateFormat=None, timestampFormat=None)

    orc(path, mode=None, partitionBy=None, compression=None)

    parquet(path, mode=None, partitionBy=None, compression=None)

## Write in HDFS

Till Spark 2.1, if we write a DF HAVING SCHEMA to Hive, it gives ERROR : xyz.csv not a SequenceFile

https://issues.apache.org/jira/browse/SPARK-9272

df.write.saveAsTable("default.mytest", format="csv", mode="overwrite", partitionBy=('age'))

Alternate ways to do this are :

### 1. When you Do not need any Partitions.

Use "CREATE TABLE db.table AS SELECT * FROM mytable". This will create an internal table in text format having same number of columns as spark's temp view say 'mytable'.

In [7]:
df.createOrReplaceTempView("mytable")

myspark.sql("DROP TABLE IF EXISTS default.simple_table")
myspark.sql("CREATE TABLE default.simple_table AS SELECT * FROM mytable")

# OR
# Create the Table in Advance and use INSERT INTO/OVERWRITE
# myspark.sql("INSERT OVERWRITE TABLE default.simple_table AS SELECT * FROM mytable")

myspark.sql("SELECT * FROM default.simple_table").show(10, False)

+------+------+---+------+----------+-------------------+
|fname |lname |age|height|dated     |timing             |
+------+------+---+------+----------+-------------------+
|naresh|jangra|30 |170.5 |2013-10-12|2013-10-12 12:35:50|
|ravi  |verma |35 |155.67|2014-10-12|2014-10-12 01:55:50|
|viren |nain  |55 |160.0 |2015-10-12|2015-10-12 09:15:50|
|bhanu |pratap|11 |180.8 |2016-10-12|2016-10-12 10:05:50|
+------+------+---+------+----------+-------------------+



### 2. When you need Dynamic Partitioned Hive Table.

i) Create the Table in Advance and load the data using df.write.save() method using required format,partition, mode etc.

Creating a Dynamic partition Table default.dynamic_part_test partitioned on "age"

In [8]:
df.createOrReplaceTempView("mytable")

myspark.conf.set("hive.exec.dynamic.partition", "true")
myspark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")

In [9]:
myspark.sql("DROP TABLE IF EXISTS default.dynamic_part_test")

myspark.sql("CREATE TABLE IF NOT EXISTS default.dynamic_part_test(\
fname string, \
lname string, \
height double, \
dated string, \
timing string )\
PARTITIONED BY (age int)\
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','")

DataFrame[]

Writing the Data to "/user/hive/warehouse/dynamic_part_test"

In [10]:
myspark.conf.set("spark.sql.shuffle.partitions",5)

df.write.save(path = "/user/hive/warehouse/dynamic_part_test", format="csv", mode="overwrite", partitionBy=("age"))

In [11]:
myspark.sql("MSCK REPAIR table default.dynamic_part_test")
myspark.sql("SELECT * FROM default.dynamic_part_test").show(10,False)

+------+------+------+----------+-------------------+---+
|fname |lname |height|dated     |timing             |age|
+------+------+------+----------+-------------------+---+
|bhanu |pratap|180.8 |2016-10-12|2016-10-12 10:05:50|11 |
|naresh|jangra|170.5 |2013-10-12|2013-10-12 12:35:50|30 |
|ravi  |verma |155.67|2014-10-12|2014-10-12 01:55:50|35 |
|viren |nain  |160.0 |2015-10-12|2015-10-12 09:15:50|55 |
+------+------+------+----------+-------------------+---+



    [768019@pxlbig14 Production EdgeNode 19:33:52 ~]$ hadoop fs -ls /user/hive/warehouse/dynamic_part_test
    Found 5 items
    -rwxrwx--x+  3 hive hive          0 2018-02-25 19:33 /user/hive/warehouse/dynamic_part_test/_SUCCESS
    drwxrwx--x+  - hive hive          0 2018-02-25 19:33 /user/hive/warehouse/dynamic_part_test/age=11
    drwxrwx--x+  - hive hive          0 2018-02-25 19:33 /user/hive/warehouse/dynamic_part_test/age=30
    drwxrwx--x+  - hive hive          0 2018-02-25 19:33 /user/hive/warehouse/dynamic_part_test/age=35
    drwxrwx--x+  - hive hive          0 2018-02-25 19:33 /user/hive/warehouse/dynamic_part_test/age=55

ii) Create the Table in Advance and load using INSERT OVERWRITE COMMAND. Make sure the order of columns is correct and partitioned Column 'age' is used in the end of SELECT command

myspark.conf.set("spark.sql.shuffle.partitions",5)

myspark.sql("INSERT OVERWRITE TABLE default.dynamic_part_test PARTITION(age)\
            SELECT fname , lname ,height ,dated,timing, age \
            FROM mytable\
            ")
myspark.sql("SELECT * FROM default.dynamic_part_test").show()

### 3. For a Static partition Table:
    
i) Create the Table in Advace and Save the Dataframe as Temp view or Table in spark and run myspark.sql("INSERT INTO hivedb.table PARTITION (name=value) select * FROM sparktable")

Creating a Static partition Table default.static_part_test partitioned on "number"

In [12]:
df.createOrReplaceTempView("mytable")

myspark.sql("DROP TABLE IF EXISTS default.static_part_test")

myspark.sql("CREATE TABLE IF NOT EXISTS default.static_part_test(\
fname string, \
lname string, \
age int, \
height double, \
dated date, \
timing string )\
PARTITIONED BY (number int)\
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','")

DataFrame[]

Inserting Data for partition number=1

In [13]:
myspark.sql("INSERT OVERWRITE TABLE default.static_part_test PARTITION (number=1) SELECT * FROM mytable")
myspark.sql("SELECT * FROM default.static_part_test where number=1 ").show()

+------+------+---+------+----------+-------------------+------+
| fname| lname|age|height|     dated|             timing|number|
+------+------+---+------+----------+-------------------+------+
|naresh|jangra| 30| 170.5|2013-10-12|2013-10-12 12:35:50|     1|
|  ravi| verma| 35|155.67|2014-10-12|2014-10-12 01:55:50|     1|
| viren|  nain| 55| 160.0|2015-10-12|2015-10-12 09:15:50|     1|
| bhanu|pratap| 11| 180.8|2016-10-12|2016-10-12 10:05:50|     1|
+------+------+---+------+----------+-------------------+------+



Inserting data for partition number=2

In [14]:
myspark.sql("INSERT OVERWRITE TABLE default.static_part_test PARTITION (number=2) SELECT * FROM mytable where fname='naresh'")
myspark.sql("SELECT * FROM default.static_part_test where number=2").show()

+------+------+---+------+----------+-------------------+------+
| fname| lname|age|height|     dated|             timing|number|
+------+------+---+------+----------+-------------------+------+
|naresh|jangra| 30| 170.5|2013-10-12|2013-10-12 12:35:50|     2|
+------+------+---+------+----------+-------------------+------+



ii) Create the Table in Advace and load the data using df.write.save() method using required format,mode etc. 
    
Note that we can not use partitionBy here which can be done only for Dynamic partition where partitoined column is part of the table.
    
We will choose the complete HDFS path (/user/hive/warehouse/static_part_test/number=3) which will include the Partition details itself.

In [15]:
myspark.conf.set("spark.sql.shuffle.partitions",5)

df.write.save(path = "/user/hive/warehouse/static_part_test/number=3", format="csv", mode="overwrite")

In [16]:
myspark.sql("MSCK REPAIR TABLE default.static_part_test")
myspark.sql("SELECT * FROM default.static_part_test where number=3").show()

+------+------+---+------+----------+-------------------+------+
| fname| lname|age|height|     dated|             timing|number|
+------+------+---+------+----------+-------------------+------+
|naresh|jangra| 30| 170.5|2013-10-12|2013-10-12 12:35:50|     3|
|  ravi| verma| 35|155.67|2014-10-12|2014-10-12 01:55:50|     3|
| viren|  nain| 55| 160.0|2015-10-12|2015-10-12 09:15:50|     3|
| bhanu|pratap| 11| 180.8|2016-10-12|2016-10-12 10:05:50|     3|
+------+------+---+------+----------+-------------------+------+



# What's Next

1) To Download this Single Notebook, Click this file in my Github Account, Copy the URL and paste in http://nbviewer.jupyter.org/. Download button will be in top right corner.

2) Open your Juypter Notebook home page and upload using "upload" Button.

3) Continue Learning from the next Notebook Spark_04_Date_Timestamp_Handling.ipynb